In [1]:
!pip install mip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.8/441.8 kB 23.7 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 1.16.0
    Uninstalling cffi-1.16.0:
      Successfully uninstalled cffi-1.16.0


In [1]:
import mip
import numpy as np

In [19]:
problem = mip.Model()

In [20]:
xs = problem.add_var_tensor((6,6,6,6,6,6,3),"xs",lb=0)
ys = problem.add_var_tensor((6,6,6,6,6,6,3),"ys",lb=0)

In [21]:
zs = problem.add_var_tensor((6,6,6,6,6,6,3,3),"zs",lb=0,ub=1)

In [22]:
us = problem.add_var_tensor((6,6,6,6,6,6,3,6,3),"us",lb=0)
vs = problem.add_var_tensor((6,6,6,6,6,6,3,6,3),"vs",lb=0)

In [23]:
# 目的関数
problem.objective = mip.minimize(mip.xsum(xs.flatten())+mip.xsum(ys.flatten())-mip.xsum(zs.flatten()))

In [29]:
prec = [
    [[1,2],[2],[]],# 0,1,2
    [[1,2],[],[1]],# 0,2,1
    [[2],[0,2],[]],# 1,0,2
    [[],[0,2],[0]],# 1,2,0
    [[1],[],[0,1]],# 2,0,1
    [[],[0],[0,1]]# 2,1,0
]

preceq = [
    [[0,1,2],[1,2],[2]],# 0,1,2
    [[0,1,2],[1],[1,2]],# 0,2,1
    [[0,2],[0,1,2],[2]],# 1,0,2
    [[0],[0,1,2],[0,2]],# 1,2,0
    [[0,1],[1],[0,1,2]],# 2,0,1
    [[0],[0,1],[0,1,2]]# 2,1,0
]

for p1 in range(6):
    for p2 in range(6):
        for p3 in range(6):
            for q1 in range(6):
                for q2 in range(6):
                    for q3 in range(6):
                        ps = [p1,p2,p3]
                        qs = [q1,q2,q3]
                        for w in range(3):
                            for f in range(3):
                                constraint = 0
                                constraint += xs[p1,p2,p3,q1,q2,q3,w] + ys[p1,p2,p3,q1,q2,q3,f] - zs[p1,p2,p3,q1,q2,q3,w,f]
                                if len(prec[ps[w]]) > 0:
                                    for f_ in prec[ps[w]][f]:
                                        constraint -= zs[p1,p2,p3,q1,q2,q3,w,f_]
                                if len(prec[qs[f]]) > 0:
                                    for w_ in prec[qs[f]][w]:
                                        constraint -= zs[p1,p2,p3,q1,q2,q3,w_,f]
                                for p_ in range(6):
                                    if w == 0:
                                        for f_ in preceq[p_][f]:
                                            constraint += us[p_,p2,p3,q1,q2,q3,w,p1,f_]
                                        for f_ in preceq[ps[w]][f]:
                                            constraint -= us[p1,p2,p3,q1,q2,q3,w,p_,f_]
                                    elif w == 1:
                                        for f_ in preceq[p_][f]:
                                            constraint += us[p1,p_,p3,q1,q2,q3,w,p2,f_]
                                        for f_ in preceq[ps[w]][f]:
                                            constraint -= us[p1,p2,p3,q1,q2,q3,w,p_,f_]
                                    elif w == 2:
                                        for f_ in preceq[p_][f]:
                                            constraint += us[p1,p2,p_,q1,q2,q3,w,p3,f_]
                                        for f_ in preceq[ps[w]][f]:
                                            constraint -= us[p1,p2,p3,q1,q2,q3,w,p_,f_]
                                for q_ in range(6):
                                    if f == 0:
                                        for w_ in preceq[q_][w]:
                                            constraint += vs[p1,p2,p3,q_,q2,q3,f,q1,w_]
                                        for w_ in preceq[qs[f]][w]:
                                            constraint -= vs[p1,p2,p3,q1,q2,q3,f,q_,w_]
                                    elif f == 1:
                                        for w_ in preceq[q_][w]:
                                            constraint += vs[p1,p2,p3,q1,q_,q3,f,q2,w_]
                                        for w_ in preceq[qs[f]][w]:
                                            constraint -= vs[p1,p2,p3,q1,q2,q3,f,q_,w_]
                                    elif f == 2:
                                        for w_ in preceq[q_][w]:
                                            constraint += vs[p1,p2,p3,q1,q2,q_,f,q3,w_]
                                        for w_ in preceq[qs[f]][w]:
                                            constraint -= vs[p1,p2,p3,q1,q2,q3,f,q_,w_]
                                problem += constraint <= 2

In [ ]:
problem.optimize()

In [ ]:
p1,p2,p3,q1,q2,q3 = 0,0,0,0,0,0

pref = [
    [[],[0],[0,1]],
    [[],[0,2],[0]],
    [[1],[],[0,1]],
    [[1,2],[],[1]],
    [[2],[0,2],[]],
    [[1,2],[2],[]]
]

for w in range(3):
    problem += mip.xsum(gs[p1,p2,p3,q1,q2,q3,w,:]) <= 1
for f in range(3):
    problem += mip.xsum(gs[p1,p2,p3,q1,q2,q3,:,f]) <= 1

for w in range(3):
    for f in range(3):
        constraint = -ts[p1,p2,p3,q1,q2,q3,w,f]-gs[p1,p2,p3,q1,q2,q3,w,f]
        for f_ in pref[ps[w]][f]:
            constraint -= gs[p1,p2,p3,q1,q2,q3,w,f_]
        for w_ in pref[qs[f]][w]:
            constraint -= gs[p1,p2,p3,q1,q2,q3,w_,f]
        problem += constraint <= -1

ps = [p1,p2,p3]
qs = [q1,q2,q3]
for w in range(3):
    for p_ in range(6):
        for f in range(3):
            constraint = 0
            flag = 0
            for f_ in pref[ps[w]][f]:
                flag = 1
                if w == 0:
                    constraint += gs[p_,p2,p3,q1,q2,q3,w,f_] - gs[p1,p2,p3,q1,q2,q3,w,f_]
                elif w == 1:
                    constraint += gs[p1,p_,p3,q1,q2,q3,w,f_] - gs[p1,p2,p3,q1,q2,q3,w,f_]
                elif w == 2:
                    constraint += gs[p1,p2,p_,q1,q2,q3,w,f_] - gs[p1,p2,p3,q1,q2,q3,w,f_]
            if flag:
                problem += constraint <= 0
for f in range(3):
    for q_ in range(6):
        for w in range(3):
            constraint = 0
            flag = 0
            for w_ in pref[qs[f]][w]:
                flag = 1
                if f == 0:
                    constraint += gs[p1,p2,p3,q_,q2,q3,w_,f] - gs[p1,p2,p3,q1,q2,q3,w_,f]
                elif f == 1:
                    constraint += gs[p1,p2,p3,q1,q_,q3,w_,f] - gs[p1,p2,p3,q1,q2,q3,w_,f]
                elif f == 2:
                    constraint += gs[p1,p2,p3,q1,q2,q_,w_,f] - gs[p1,p2,p3,q1,q2,q3,w_,f]
            if flag:
                problem += constraint <= 0

In [ ]:
problem.vars[0].name

'gs_0_0_0_0_0_0_0_0'

In [ ]:
constraints = {}
names_ = ["gs_0_0_0_0_0_0_0_0","gs_0_0_0_0_0_0_0_1","gs_0_0_0_0_0_0_0_2",
         "gs_0_0_0_0_0_0_1_0","gs_0_0_0_0_0_0_1_1","gs_0_0_0_0_0_0_1_2",
         "gs_0_0_0_0_0_0_2_0","gs_0_0_0_0_0_0_2_1","gs_0_0_0_0_0_0_2_2",
         "gs_4_1_1_0_3_5_0_0","gs_4_1_1_0_3_5_0_1","gs_4_1_1_0_3_5_0_2",
         "gs_4_1_1_0_3_5_1_0","gs_4_1_1_0_3_5_1_1","gs_4_1_1_0_3_5_1_2",
         "gs_4_1_1_0_3_5_2_0","gs_4_1_1_0_3_5_2_1","gs_4_1_1_0_3_5_2_2"]
for var in names_:
    constraints[var] = []
constraints["b"] = []

In [ ]:
len(index_list)

4059072

In [ ]:
import random

In [ ]:
random.randint(0,5)

NameError: name 'random' is not defined

In [ ]:
for constr in problem.constrs:
    names = list(x.name for x in constr.expr.expr)
    names.append("b")
    for var in constraints:
        if var not in names:
            constraints[var].append(0)
    for x in constr.expr.expr:
        if x.name in names_:
            constraints[x.name].append(constr.expr.expr[x])
    constraints["b"].append(constr.rhs)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(constraints,index=index_list)
df = df[df.iloc[:,:-1].abs().sum(axis=1) != 0]

In [ ]:
df = df[df.iloc[:,:-1].abs().sum(axis=1) != 0]

In [ ]:
df.shape

(270, 19)

In [ ]:
len(problem.constrs)

4059072

In [ ]:
len(index_list)

4059072

In [ ]:
df = df.iloc[:,:9]

In [ ]:
df.to_csv("constraints33.csv")

In [ ]:
# fractionally stable
pref = [
    [[],[0],[0,1]],
    [[],[0,2],[0]],
    [[1],[],[0,1]],
    [[1,2],[],[1]],
    [[2],[0,2],[]],
    [[1,2],[2],[]]
]

for p1 in range(6):
    for p2 in range(6):
        for p3 in range(6):
            for q1 in range(6):
                for q2 in range(6):
                    for q3 in range(6):
                        ps = [p1,p2,p3]
                        qs = [q1,q2,q3]
                        for w in range(3):
                            for f in range(3):
                                constraint = -ts[p1,p2,p3,q1,q2,q3,w,f]-gs[p1,p2,p3,q1,q2,q3,w,f]
                                for f_ in pref[ps[w]][f]:
                                    constraint -= gs[p1,p2,p3,q1,q2,q3,w,f_]
                                for w_ in pref[qs[f]][w]:
                                    constraint -= gs[p1,p2,p3,q1,q2,q3,w_,f]
                                problem += constraint <= -1
                                index_list.append(f"z_{w}{f}_{p1}{p2}{p3}{q1}{q2}{q3}")

In [ ]:
# strategy proofness
for p1 in range(6):
    for p2 in range(6):
        for p3 in range(6):
            for q1 in range(6):
                for q2 in range(6):
                    for q3 in range(6):
                        ps = [p1,p2,p3]
                        qs = [q1,q2,q3]
                        for w in range(3):
                            for p_ in range(6):
                                if p_ != ps[w]:
                                    for f in range(3):
                                        constraint = 0
                                        flag = 0
                                        for f_ in pref[ps[w]][f]:
                                            flag = 1
                                            if w == 0:
                                                constraint += gs[p_,p2,p3,q1,q2,q3,w,f_] - gs[p1,p2,p3,q1,q2,q3,w,f_]
                                            elif w == 1:
                                                constraint += gs[p1,p_,p3,q1,q2,q3,w,f_] - gs[p1,p2,p3,q1,q2,q3,w,f_]
                                            elif w == 2:
                                                constraint += gs[p1,p2,p_,q1,q2,q3,w,f_] - gs[p1,p2,p3,q1,q2,q3,w,f_]
                                        if flag:
                                            problem += constraint <= 0
                                            index_list.append(f"u_{w}{f}_{p_}_{p1}{p2}{p3}{q1}{q2}{q3}")
                        for f in range(3):
                            for q_ in range(6):
                                if q_ != qs[f]:
                                    for w in range(3):
                                        constraint = 0
                                        flag = 0
                                        for w_ in pref[qs[f]][w]:
                                            flag = 1
                                            if f == 0:
                                                constraint += gs[p1,p2,p3,q_,q2,q3,w_,f] - gs[p1,p2,p3,q1,q2,q3,w_,f]
                                            elif f == 1:
                                                constraint += gs[p1,p2,p3,q1,q_,q3,w_,f] - gs[p1,p2,p3,q1,q2,q3,w_,f]
                                            elif f == 2:
                                                constraint += gs[p1,p2,p3,q1,q2,q_,w_,f] - gs[p1,p2,p3,q1,q2,q3,w_,f]
                                        if flag:
                                            problem += constraint <= 0
                                            index_list.append(f"v_{w}{f}_{q_}_{p1}{p2}{p3}{q1}{q2}{q3}")

In [ ]:
problem.optimize()

In [ ]:
# fairness
change = [
    [[[1,0,2],[2,1,0]],[0,2,1]],
    [[[1,2,0],[2,0,1]],[0,1,2]],
    [[[0,1,2],[1,2,0]],[2,0,1]],
    [[[0,2,1],[1,0,2]],[2,1,0]],
    [[[2,1,0],[0,2,1]],[1,0,2]],
    [[[2,0,1],[0,1,2]],[1,2,0]]
]

for p1 in range(6):
    for p2 in range(6):
        for p3 in range(6):
            for q1 in range(6):
                for q2 in range(6):
                    for q3 in range(6):
                        ps = [p1,p2,p3]
                        qs = [q1,q2,q3]
                        # for w in range(3):
                        #     for f in range(3):
                        #         problem += gs[p1][p2][p3][q1][q2][q3][w][f] == gs[q1][q2][q3][p1][p2][p3][f][w]
                        for w1 in range(2):
                            for w2 in range(w1,3):
                                w3 = 3-w1-w2
                                for f in range(3):
                                    if w3 == 2:
                                        problem += gs[p1][p2][p3][q1][q2][q3][w1][f] == gs[p2][p1][p3][change[q1][w1][w2-1]][change[q2][w1][w2-1]][change[q3][w1][w2-1]][w2][f]
                                        problem += gs[p1][p2][p3][q1][q2][q3][w2][f] == gs[p2][p1][p3][change[q1][w1][w2-1]][change[q2][w1][w2-1]][change[q3][w1][w2-1]][w1][f]
                                        problem += gs[p1][p2][p3][q1][q2][q3][w3][f] == gs[p2][p1][p3][change[q1][w1][w2-1]][change[q2][w1][w2-1]][change[q3][w1][w2-1]][w3][f]
                                    elif w3 == 1:
                                        problem += gs[p1][p2][p3][q1][q2][q3][w1][f] == gs[p3][p2][p1][change[q1][w1][w2-1]][change[q2][w1][w2-1]][change[q3][w1][w2-1]][w2][f]
                                        problem += gs[p1][p2][p3][q1][q2][q3][w2][f] == gs[p3][p2][p1][change[q1][w1][w2-1]][change[q2][w1][w2-1]][change[q3][w1][w2-1]][w1][f]
                                        problem += gs[p1][p2][p3][q1][q2][q3][w3][f] == gs[p3][p2][p1][change[q1][w1][w2-1]][change[q2][w1][w2-1]][change[q3][w1][w2-1]][w3][f]
                                    elif w3 == 0:
                                        problem += gs[p1][p2][p3][q1][q2][q3][w1][f] == gs[p1][p3][p2][change[q1][w1][w2-1]][change[q2][w1][w2-1]][change[q3][w1][w2-1]][w2][f]
                                        problem += gs[p1][p2][p3][q1][q2][q3][w2][f] == gs[p1][p3][p2][change[q1][w1][w2-1]][change[q2][w1][w2-1]][change[q3][w1][w2-1]][w1][f]
                                        problem += gs[p1][p2][p3][q1][q2][q3][w3][f] == gs[p1][p3][p2][change[q1][w1][w2-1]][change[q2][w1][w2-1]][change[q3][w1][w2-1]][w3][f]
                        for f1 in range(2):
                            for f2 in range(f1,3):
                                f3 = 3-f1-f2
                                for w in range(3):
                                    if f3 == 2:
                                        problem += gs[p1][p2][p3][q1][q2][q3][w][f1] == gs[change[p1][f1][f2-1]][change[p2][f1][f2-1]][change[p3][f1][f2-1]][q2][q1][q3][w][f2]
                                        problem += gs[p1][p2][p3][q1][q2][q3][w][f2] == gs[change[p1][f1][f2-1]][change[p2][f1][f2-1]][change[p3][f1][f2-1]][q2][q1][q3][w][f1]
                                        problem += gs[p1][p2][p3][q1][q2][q3][w][f3] == gs[change[p1][f1][f2-1]][change[p2][f1][f2-1]][change[p3][f1][f2-1]][q2][q1][q3][w][f3]
                                    elif f3 == 1:
                                        problem += gs[p1][p2][p3][q1][q2][q3][w][f1] == gs[change[p1][f1][f2-1]][change[p2][f1][f2-1]][change[p3][f1][f2-1]][q3][q2][q1][w][f3]
                                        problem += gs[p1][p2][p3][q1][q2][q3][w][f2] == gs[change[p1][f1][f2-1]][change[p2][f1][f2-1]][change[p3][f1][f2-1]][q3][q2][q1][w][f2]
                                        problem += gs[p1][p2][p3][q1][q2][q3][w][f3] == gs[change[p1][f1][f2-1]][change[p2][f1][f2-1]][change[p3][f1][f2-1]][q3][q2][q1][w][f1]
                                    elif f3 == 0:
                                        problem += gs[p1][p2][p3][q1][q2][q3][w][f1] == gs[change[p1][f1][f2-1]][change[p2][f1][f2-1]][change[p3][f1][f2-1]][q1][q3][q2][w][f1]
                                        problem += gs[p1][p2][p3][q1][q2][q3][w][f2] == gs[change[p1][f1][f2-1]][change[p2][f1][f2-1]][change[p3][f1][f2-1]][q1][q3][q2][w][f3]
                                        problem += gs[p1][p2][p3][q1][q2][q3][w][f3] == gs[change[p1][f1][f2-1]][change[p2][f1][f2-1]][change[p3][f1][f2-1]][q1][q3][q2][w][f2]

In [ ]:
result = problem.solve()

KeyboardInterrupt: 

In [ ]:
for p1 in range(2):
    for p2 in range(2):
        for q1 in range(2):
            for q2 in range(2):
                for w in range(2):
                    for f in range(2):
                        print(f"g_{w}{f}_{p1}{p2}{q1}{q2}:{gs[p1][p2][q1][q2][w][f].value()}")

g_00_0000:1.0
g_01_0000:0.0
g_10_0000:0.0
g_11_0000:1.0
g_00_0001:0.0
g_01_0001:1.0
g_10_0001:1.0
g_11_0001:0.0
g_00_0010:1.0
g_01_0010:0.0
g_10_0010:0.0
g_11_0010:1.0
g_00_0011:0.0
g_01_0011:1.0
g_10_0011:1.0
g_11_0011:0.0
g_00_0100:0.0
g_01_0100:1.0
g_10_0100:1.0
g_11_0100:0.0
g_00_0101:0.0
g_01_0101:1.0
g_10_0101:1.0
g_11_0101:0.0
g_00_0110:1.0
g_01_0110:0.0
g_10_0110:0.0
g_11_0110:1.0
g_00_0111:0.0
g_01_0111:1.0
g_10_0111:1.0
g_11_0111:0.0
g_00_1000:1.0
g_01_1000:0.0
g_10_1000:0.0
g_11_1000:1.0
g_00_1001:0.0
g_01_1001:1.0
g_10_1001:1.0
g_11_1001:0.0
g_00_1010:1.0
g_01_1010:0.0
g_10_1010:0.0
g_11_1010:1.0
g_00_1011:1.0
g_01_1011:0.0
g_10_1011:0.0
g_11_1011:1.0
g_00_1100:0.0
g_01_1100:1.0
g_10_1100:1.0
g_11_1100:0.0
g_00_1101:0.0
g_01_1101:1.0
g_10_1101:1.0
g_11_1101:0.0
g_00_1110:1.0
g_01_1110:0.0
g_10_1110:0.0
g_11_1110:1.0
g_00_1111:1.0
g_01_1111:0.0
g_10_1111:0.0
g_11_1111:1.0
